# Entrenamiento usando las técnicas de HuggingFace

### Chequear GPU

(Error después porque hay que usar un tipo de modelo distinto)

In [57]:
#from pynvml import *
#
#
#def print_gpu_utilization():
#    nvmlInit()
#    handle = nvmlDeviceGetHandleByIndex(0)
#    info = nvmlDeviceGetMemoryInfo(handle)
#    print(f"GPU memory occupied: {info.used//1024**2} MB.")
#
#
#def print_summary(result):
#    print(f"Time: {result.metrics['train_runtime']:.2f}")
#    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
#    print_gpu_utilization()

In [58]:
#print_gpu_utilization()

In [59]:
#import torch
#
#torch.ones((1, 1)).to("cuda")
#
#print_gpu_utilization()

In [60]:
#torch.cuda.is_available()

### Función para convertir labels en dataset de tweets

In [61]:
def convertir_labels(train_data, val_data):
    MIN_VOTOS = 2

    if MIN_VOTOS == 3:
        # Considerar que un dato pertenece solo si las 3 personas pusieron que pertenece
        train_data['Odio'] = np.where(train_data['Odio'] == 3, 1, 0)
        val_data['Odio'] = np.where(val_data['Odio'] == 3, 1, 0)
        train_data['Mujeres'] = np.where(train_data['Mujeres'] == 3, 1, 0)
        val_data['Mujeres'] = np.where(val_data['Mujeres'] == 3, 1, 0)
        train_data['Comunidad LGBTQ+'] = np.where(train_data['Comunidad LGBTQ+'] == 3, 1, 0)
        val_data['Comunidad LGBTQ+'] = np.where(val_data['Comunidad LGBTQ+'] == 3, 1, 0)
        train_data['Comunidades Migrantes'] = np.where(train_data['Comunidades Migrantes'] == 3, 1, 0)
        val_data['Comunidades Migrantes'] = np.where(val_data['Comunidades Migrantes'] == 3, 1, 0)
        train_data['Pueblos Originarios'] = np.where(train_data['Pueblos Originarios'] == 3, 1, 0)
        val_data['Pueblos Originarios'] = np.where(val_data['Pueblos Originarios'] == 3, 1, 0)   
    elif MIN_VOTOS == 2:
        train_data['Odio'] = np.where((train_data['Odio'] == 3) | (train_data['Odio'] == 2), 1, 0)
        val_data['Odio'] = np.where((val_data['Odio'] == 3) | (val_data['Odio'] == 2), 1, 0)
        train_data['Mujeres'] = np.where((train_data['Mujeres'] == 3) | (train_data['Mujeres'] == 2), 1, 0)
        val_data['Mujeres'] = np.where((val_data['Mujeres'] == 3) | (val_data['Mujeres'] == 2), 1, 0)
        train_data['Comunidad LGBTQ+'] = np.where((train_data['Comunidad LGBTQ+'] == 3) | (train_data['Comunidad LGBTQ+'] == 2), 1, 0)
        val_data['Comunidad LGBTQ+'] = np.where((val_data['Comunidad LGBTQ+'] == 3) | (val_data['Comunidad LGBTQ+'] == 2), 1, 0)
        train_data['Comunidades Migrantes'] = np.where((train_data['Comunidades Migrantes'] == 3) | (train_data['Comunidades Migrantes'] == 2), 1, 0)
        val_data['Comunidades Migrantes'] = np.where((val_data['Comunidades Migrantes'] == 3) | (val_data['Comunidades Migrantes'] == 2), 1, 0)
        train_data['Pueblos Originarios'] = np.where((train_data['Pueblos Originarios'] == 3) | (train_data['Pueblos Originarios'] == 2), 1, 0)
        val_data['Pueblos Originarios'] = np.where((val_data['Pueblos Originarios'] == 3) | (val_data['Pueblos Originarios'] == 2), 1, 0)

    return train_data, val_data

### Funcion de limpieza de texto

In [62]:
import unidecode
import spacy_spanish_lemmatizer
import spacy
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import spacy_spanish_lemmatizer

stemmer = SnowballStemmer("spanish")
nlp = spacy.load("es_core_news_lg")
#nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")
stop_words = stopwords.words("spanish")

def clean_text(text, len_words):
    TEXT_CLEANING_RE = "@\S+" # separado porque solo agarraba la primera etiqueta
    TEXT_CLEANING_RE2 = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
    TEXT_CLEANING_RE_EXTRA = "[^\w\s]"
    if len_words == 1:
        text = re.sub(r'\b\w{1}\b', '', str(text).lower()).strip()
    elif len_words == 2:
        text = re.sub(r'\b\w{1,2}\b', '', str(text).lower()).strip()
    elif len_words == 3:
        text = re.sub(r'\b\w{1,3}\b', '', str(text).lower()).strip()
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    text = re.sub(TEXT_CLEANING_RE2, ' ', str(text).lower()).strip()
    text = re.sub(TEXT_CLEANING_RE_EXTRA, ' ', str(text).lower()).strip()
    return text

def preprocess(text,cleaning=True, stopwords=True, stemming=False, lemmatizer=False, len_words=1):
    text = unidecode.unidecode(text)
    if cleaning:
        text = clean_text(text, len_words)
    tokens = []
    for token in text.split():
        if (not stopwords) or (stopwords and (token not in stop_words)):
            if stemming:
                tokens.append(stemmer.stem(token))
            elif lemmatizer:
                doc = nlp(token)
                tokens.append(unidecode.unidecode(doc[0].lemma_))
            else:
                tokens.append(token)    
    text2 = " ".join(tokens)
    if cleaning:
        text2 = clean_text(text2, len_words)
    return text2

## Cargar datasets

Notar que estos son de tipo "Dataset", un tipo que se usa en HuggingFace comunmente.

In [63]:
import pandas as pd
from datasets import Dataset
import numpy as np

csv = pd.read_csv("data/tweets_train.csv")

TRAIN_SIZE = 0.85

train_data = csv.sample(frac=1).reset_index(drop=True)
train_data, val_data = train_data.iloc[:int(TRAIN_SIZE*len(train_data)), :], train_data.iloc[int(TRAIN_SIZE*len(train_data)):, :]
train_data, val_data = convertir_labels(train_data, val_data)
train_data.drop(columns=['Unnamed: 0', 'tweet_id', 'author_id', 'conversation_id'], inplace=True)
val_data.drop(columns=['Unnamed: 0', 'tweet_id', 'author_id', 'conversation_id'], inplace=True)

train_data['text'] = train_data['text'].apply(lambda x: preprocess(x, cleaning=True, stopwords=False, stemming=False, lemmatizer=False, len_words=0))
val_data['text'] = val_data['text'].apply(lambda x: preprocess(x, cleaning=True, stopwords=False, stemming=False, lemmatizer=False, len_words=0))

# convertir a dataset de huggingface
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

print(train_dataset)
print(train_dataset[1])

Dataset({
    features: ['text', 'Odio', 'Mujeres', 'Comunidad LGBTQ+', 'Comunidades Migrantes', 'Pueblos Originarios'],
    num_rows: 1917
})
{'text': 'choro', 'Odio': 0, 'Mujeres': 0, 'Comunidad LGBTQ+': 0, 'Comunidades Migrantes': 0, 'Pueblos Originarios': 0}


#### Convertir columnas en labels para losd datasets

In [64]:
labels = ['Odio', 'Mujeres', 'Comunidad LGBTQ+', 'Comunidades Migrantes', 'Pueblos Originarios']
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
labels

['Odio',
 'Mujeres',
 'Comunidad LGBTQ+',
 'Comunidades Migrantes',
 'Pueblos Originarios']

## Tokenizar

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence"], truncation=True)

def preprocess_data(examples):
    text = examples['text']
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128)
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding['labels'] = labels_matrix
    return encoding

encoded_train = train_dataset.map(preprocess_data, batched=True, remove_columns=labels)
encoded_val = val_dataset.map(preprocess_data, batched=True, remove_columns=labels)



encoded_train.set_format("torch")
encoded_val.set_format("torch")

In [66]:
print(encoded_train[2])
print(encoded_train[2].keys())

{'text': 'dona bachelet nos dejo el pais infectado con tanto haitiano son sucios', 'input_ids': tensor([  101,  1274,  1161,   171, 12804,  5765,  1185,  1116,  1260,  5077,
         8468,   185, 15837,  1107, 11916,  9359, 14255, 15925,  2430,  5871,
        17030,  7428,  1488, 28117,  8174,  1116,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    

## Preparar dataset y modelo

In [67]:
from transformers import AutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', num_labels=5, 
                                                             problem_type="multi_label_classification", id2label=id2label, 
                                                             label2id=label2id)


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

In [68]:
batch_size = 8
metric_name = "f1"

In [69]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [70]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [71]:
encoded_train[0]['labels'].type()    

'torch.FloatTensor'

In [72]:
encoded_train['input_ids'][0]

tensor([  101,   175,  8558, 21359, 23015,  1279,   178,  1197,   170,  2495,
         1508,  1161,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [73]:
outputs = model(input_ids=encoded_train['input_ids'][0].unsqueeze(0), labels=encoded_train[0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7357, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0955, -0.0559,  0.2567, -0.1156,  0.2113]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Entrenar

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

## Predecir